In [1]:
import numpy as np

# 前処理にはsklearnライブラリを使っていきます
from sklearn import preprocessing

# データを読み込みます
raw_csv_data = np.loadtxt('Audiobooks-data.csv',delimiter=',')

# 最初のIDの列と、ターゲットである最後の列を取り除いた上で変数に代入します
unscaled_inputs_all = raw_csv_data[:,1:-1]

# 最後の列のデータをターゲットとして変数に代入していきます
targets_all = raw_csv_data[:,-1]

In [2]:
targets_all.shape

(14084,)

In [3]:
# ターゲットの数を数えます
num_one_targets = int(np.sum(targets_all))

# 数を数えるための変数を定義します
zero_targets_counter = 0

# バランスを取る上で不要なデータを入れるためのリストを定義します
indices_to_remove = []

# ターゲットの数を数え、その数を超える0のデータがあった場合はリストにその値をいれていきます
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# 入力とターゲットを入れるための変数を定義します
# また、バランスが取れていないデータを削除していきます
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [4]:
num_one_targets

2237

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)


In [6]:
# データをバッチに分けていくことから、何らかの規則性が生まれないようにデータをシャッフルしていきます
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# シャッフルしたデータを変数に入れていきます
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]




In [7]:
# データの数を数えます
samples_count = shuffled_inputs.shape[0]

# 80-10-10に分けることを前提として、訓練用と検証用のデータの数を定義していきます
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# テスト用のデータの数を定義します
test_samples_count = samples_count - train_samples_count - validation_samples_count

# 訓練用データを作成します
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# 検証用データを作成します
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# テスト用データを作成します
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


# 作成したデータの中の１と０の割合を確認します
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1800.0 3579 0.5029337803855826
222.0 447 0.4966442953020134
215.0 448 0.4799107142857143


In [8]:
# Ssavezメソッドを使ってデータを保存していきます
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [9]:
# 新たに一からコードを書いていきますので、再度ライブラリをインポートします
import numpy as np
import tensorflow as tf

In [10]:
# まずは訓練データを一時的に読み込みます
npz = np.load('Audiobooks_data_train.npz')

# 訓練用のデータを入力とターゲットに分けて変数に代入していきます
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

# 検証用データを一時的に読み込みます
npz = np.load('Audiobooks_data_validation.npz')
# 訓練用のデータを入力とターゲットに分けて変数に代入していきます
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# テストデータを一時的に読み込みます
npz = np.load('Audiobooks_data_test.npz')
# テスト用のデータを入力とターゲットに分けて変数に代入していきます
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

C:\Users\fujir\AppData\Local\Temp\ipykernel_17028\2274972335.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
C:\Users\fujir\AppData\Local\Temp\ipykernel_17028\2274972335.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org

In [11]:
# 入力の数と出力の数を定義します
input_size = 10
output_size = 2
# 隠れ層のユニットの数を定義します
hidden_layer_size = 30
    
# モデルを作成していきます
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1番目の隠れ層
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2番目の隠れ層
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 3番目の隠れ層
     tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 4番目の隠れ層
   
    tf.keras.layers.Dense(output_size, activation='softmax') # 出力層
])


# 損失関数と最適化アルゴリズムを定義していきます
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 訓練

# バッチサイズの設定
batch_size = 100

# 繰り返し回数の設定
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
# モデルにデータを入れていきます
model.fit(train_inputs, # 訓練データの入力
          train_targets, # 訓練データのターゲット
          batch_size=batch_size, # バッチサイズ
          epochs=max_epochs, # 繰り返し回数
            callbacks=[early_stopping], # アーリーストッピング
          validation_data=(validation_inputs, validation_targets), # 検証データの指定
          verbose = 2 # 結果の見た目を変更
          )  

Epoch 1/100
36/36 - 0s - loss: 0.6223 - accuracy: 0.6329 - val_loss: 0.5613 - val_accuracy: 0.6801
Epoch 2/100
36/36 - 0s - loss: 0.5030 - accuracy: 0.7348 - val_loss: 0.4718 - val_accuracy: 0.7293
Epoch 3/100
36/36 - 0s - loss: 0.4332 - accuracy: 0.7628 - val_loss: 0.4300 - val_accuracy: 0.7539
Epoch 4/100
36/36 - 0s - loss: 0.3963 - accuracy: 0.7924 - val_loss: 0.4098 - val_accuracy: 0.7517
Epoch 5/100
36/36 - 0s - loss: 0.3740 - accuracy: 0.7994 - val_loss: 0.3989 - val_accuracy: 0.7696
Epoch 6/100
36/36 - 0s - loss: 0.3605 - accuracy: 0.8013 - val_loss: 0.4116 - val_accuracy: 0.7539
Epoch 7/100
36/36 - 0s - loss: 0.3488 - accuracy: 0.8145 - val_loss: 0.3767 - val_accuracy: 0.7897
Epoch 8/100
36/36 - 0s - loss: 0.3423 - accuracy: 0.8170 - val_loss: 0.3733 - val_accuracy: 0.7830
Epoch 9/100
36/36 - 0s - loss: 0.3401 - accuracy: 0.8156 - val_loss: 0.3882 - val_accuracy: 0.7718
Epoch 10/100
36/36 - 0s - loss: 0.3325 - accuracy: 0.8212 - val_loss: 0.4105 - val_accuracy: 0.7808


In [12]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3653 - accuracy: 0.7946


In [13]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.37. Test accuracy: 79.46%
